#### **1. 导入模块**

导入 Python 标准库和本项目自定义库

In [1]:
# 标准库
import os
import sys
import time

# 将上级目录加入系统路径
# 以便导入项目自定义库
sys.path.append(os.path.abspath('..'))

# 自定义库
from src.utils import *
from src.annotator.sem_tagger import *

#### **2. 读取语料**

读取 TSV 格式的平行语料库

In [2]:
# 指定语料库路径
data_file = '../data/raw/test_data.tsv'

# 读取 TSV 格式语料
# 数据存储于 DataFrame 对象
data = load_data(data_file)
print(f'成功读取 {len(data)} 条平行句对')

# 预览数据
# 第一列：汉语原文，选自《鹿鼎记》
# 第二列：英语译文，选自《The Deer and The Cauldron》（闵福德译）
print('数据前 5 行如下：')
data.head()

成功读取 10 条平行句对
数据前 5 行如下：


,source,target
1,江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。,Along a coastal road somewhere south of the Ya...
2,前面三辆囚车中分别监禁的是三个男子，都作书生打扮，一个是白发老者，两个是中年人。,In each of the first three carts a single male...
3,后面四辆囚车中坐的是女子，最后一辆囚车中是个少妇，怀中抱着个女婴。,"The four rear carts were occupied by women, th..."
4,女婴啼哭不休。 她母亲温言相呵，女婴只是大哭。,The little girl was crying in a continuous wai...
5,囚车旁一清兵恼了，伸腿在车上踢了一脚，喝道：“再哭，再哭！,"One of the soldiers marching alongside, irrita..."


#### **3. 大模型英文语义标注：USAS 赋码集**

In [3]:
# === 加载模型：LLM USAS 赋码集 ===

# 语种：英语
lang = 'english'

# 赋码集：USAS
# https://ucrel.lancs.ac.uk/usas/
# https://ucrel.lancs.ac.uk/usas/usas_guide.pdf
tagset = 'usas'

# 标注模式：LLM API
mode = 'llm'

# 大模型：deepseek-v3.2 | kimi-k2.5 | glm-4.7 | qwen3-max
llm_model = 'deepseek-v3.2'

# 思考模式：关闭
enable_thinking = False

# 登录阿里云百炼平台：https://bailian.console.aliyun.com/
# 申请调用大模型服务的 API 账号
# 并在 llm_corpus_annotation/config 文件中设置 LLM_API_KEY=sk-********
t0 = time.time()
llm_en_usas_tagger = SEMTagger(
    lang=lang,
    tagset=tagset,
    mode=mode,
    llm_model=llm_model,
    enable_thinking=enable_thinking,
)
t1 = time.time()
print('LLM USAS 英文语义标注模型加载完毕！')
print(f'加载耗时：{t1-t0:.2f}s')

LLM USAS 英文语义标注模型加载完毕！
加载耗时：1.81s


In [4]:
# === 标注译文：LLM USAS 赋码集 ===

# 注意：
# 为节省 API 调用成本
# 大模型生成内容保存于本地缓存 data/llm_cache
# 完成首次调用后，再次调用只需从本地数据库读取生成结果
# 因此，再次运行程序，时间显示为 0.0 秒
llm_en_usas_annos = annotate_data(
    data['target'],
    llm_en_usas_tagger,
)

# 打印标注结果
print("\n=== LLM USAS 英文语义标注结果预览 ===")
display_anno(llm_en_usas_annos[0])

Semantic Tagging: 100%|███████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 414.98it/s]


=== LLM USAS 英文语义标注结果预览 ===

[ID]: 00001
Along a coastal road somewhere south of the Yangtze River, a detachment of soldiers, each of them armed with a halberd, was escorting a line of seven prison carts, trudging northwards in the teeth of a bitter wind.
--------------------------------------------------------------------------------
[('Along', 'M6'), ('a', 'Z5'), ('coastal road', 'M3'), ('somewhere', 'M7'), ('south', 'M6'), ('of', 'Z5'), ('the', 'Z5'), ('Yangtze River', 'W3/Z2'), (',', 'PUNCT'), ('a', 'Z5'), ('detachment', 'S5'), ('of', 'Z5'), ('soldiers', 'G3/S2'), (',', 'PUNCT'), ('each', 'N5.1-'), ('of', 'Z5'), ('them', 'Z8'), ('armed', 'G3'), ('with', 'Z5'), ('a', 'Z5'), ('halberd', 'G3'), (',', 'PUNCT'), ('was', 'A3+'), ('escorting', 'S8+'), ('a', 'Z5'), ('line', 'N4'), ('of', 'Z5'), ('seven', 'N1'), ('prison carts', 'G2.1-/M3'), (',', 'PUNCT'), ('trudging', 'M1'), ('northwards', 'M6'), ('in the teeth of', 'A9-'), ('a', 'Z5'), ('bitter', 'O4.6-'), ('wind', 'W4'), ('.', 'PUNCT')

#### **4. 英文语义标注模型对比：LLM & PyMUSAS**

In [5]:
# === 加载模型：PyUSAS USAS 赋码集 ===

# 语种：英语
lang = 'english'

# 赋码集：USAS
# https://ucrel.lancs.ac.uk/usas/
# https://ucrel.lancs.ac.uk/usas/usas_guide.pdf
tagset = 'usas'

# 标注模式：pyMUSAS 本地模型
mode = 'local'

# PyMUSAS 模型：Rule-based Tagger
# https://ucrel.github.io/pymusas/usage/how_to/tag_text_with/rule_based_tagger
# https://github.com/UCREL/Multilingual-USAS/tree/master/English
# 首次加载需从网络将模型下载至本地
t0 = time.time()
pymusas_en_usas_tagger = SEMTagger(
    lang=lang,
    tagset=tagset,
    mode=mode,
)
t1 = time.time()
print('PyMUSAS USAS 英文语义标注标注模型加载完毕！')
print(f'加载耗时：{t1-t0:.2f}s')

PyMUSAS USAS 英文语义标注标注模型加载完毕！
加载耗时：7.73s


In [6]:
# === 标注译文：PyUSAS USAS 赋码集 ===

pymusas_en_usas_annos = annotate_data(
    data['target'],
    pymusas_en_usas_tagger,
)

# 打印标注结果
print("\n=== PyMUSAS USAS 英文语义标注标注结果预览 ===")
display_anno(pymusas_en_usas_annos[0])

Semantic Tagging: 100%|████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 25.94it/s]


=== PyMUSAS USAS 英文语义标注标注结果预览 ===

[ID]: 00001
Along a coastal road somewhere south of the Yangtze River, a detachment of soldiers, each of them armed with a halberd, was escorting a line of seven prison carts, trudging northwards in the teeth of a bitter wind.
--------------------------------------------------------------------------------
[('Along', 'Z5'), ('a', 'Z5'), ('coastal', 'W3/M4'), ('road', 'M3'), ('somewhere', 'M6'), ('south', 'M6'), ('of', 'Z5'), ('the', 'Z5'), ('Yangtze River', 'Z1'), (',', 'PUNCT'), ('a', 'Z5'), ('detachment', 'X5.2-'), ('of', 'Z5'), ('soldiers', 'G3/S2'), (',', 'PUNCT'), ('each', 'N5.1+'), ('of', 'Z5'), ('them', 'Z8'), ('armed', 'G3'), ('with', 'Z5'), ('a', 'Z5'), ('halberd', 'Z99'), (',', 'PUNCT'), ('was', 'A3+'), ('escorting', 'S3.1'), ('a', 'Z5'), ('line', 'O4.4'), ('of', 'Z5'), ('seven', 'N1'), ('prison', 'G2.1/H1'), ('carts', 'M3'), (',', 'PUNCT'), ('trudging', 'M1'), ('northwards', 'M6'), ('in', 'Z5'), ('the', 'Z5'), ('teeth', 'B1'), ('of', 'Z5')

In [7]:
# === 模型对比：LLM USAS & PyMUSAS USAS ===

print('英文语义标注模型对比（USAS 赋码集）：LLM & PyMUSAS')
compare_annos(
    llm_en_usas_annos,
    pymusas_en_usas_annos,
    annos_1_name="LLM",
    annos_2_name="PyMUAS",
)

英文语义标注模型对比（USAS 赋码集）：LLM & PyMUSAS

[ID]: 00001
Along a coastal road somewhere south of the Yangtze River, a detachment of soldiers, each of them armed with a halberd, was escorting a line of seven prison carts, trudging northwards in the teeth of a bitter wind.
--------------------------------------------------------------------------------
Jaccard: 0.357
LLM: {('detachment', 'S5'), ('in the teeth of', 'A9-'), ('bitter', 'O4.6-'), ('line', 'N4'), ('halberd', 'G3'), ('Yangtze River', 'W3/Z2'), ('prison carts', 'G2.1-/M3'), ('coastal road', 'M3'), ('escorting', 'S8+'), ('each', 'N5.1-'), ('somewhere', 'M7'), ('Along', 'M6')}
PyMUAS: {('teeth', 'B1'), ('coastal', 'W3/M4'), ('Along', 'Z5'), ('carts', 'M3'), ('halberd', 'Z99'), ('each', 'N5.1+'), ('somewhere', 'M6'), ('line', 'O4.4'), ('prison', 'G2.1/H1'), ('bitter', 'X3.1'), ('escorting', 'S3.1'), ('detachment', 'X5.2-'), ('road', 'M3'), ('Yangtze River', 'Z1'), ('in', 'Z5')}

[ID]: 00002
In each of the first three carts a single male pr